## Data Ingestion Module

import os

In [1]:
import os
%pwd

'd:\\MLOPS\\TextSummarizer\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\MLOPS\\TextSummarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

    

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories


In [5]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger


In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"File downloaded at {filename}")
        else:
            logger.info(f"File already exists")
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info(f"File extracted at {self.config.unzip_dir}")


In [9]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-01-02 12:25:13,774] INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 12:25:14,232] INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 12:25:14,267] INFO: common: creating directory at: artifacts]
[2025-01-02 12:25:14,270] INFO: common: creating directory at: artifacts/data_ingestion]
[2025-01-02 12:25:26,289] INFO: 1252426548: File downloaded at artifacts/data_ingestion/data.zip]
[2025-01-02 12:25:27,497] INFO: 1252426548: File extracted at artifacts/data_ingestion]
